Library Imports

In [ ]:
import numpy as np
import pandas as pd
from sklearn import datasets
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point
from sklearn.decomposition import PCA
from sklearn.base import BaseEstimator, TransformerMixin
from utils.utils import encode
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report, confusion_matrix, mean_squared_error, f1_score


Import Libraries

In [ ]:
digits = datasets.load_digits()

df_digits = pd.DataFrame(digits.data, columns=[f'Pixel_{i}' for i in range(digits.data.shape[1])])
df_digits['Target'] = digits.target


housing = fetch_california_housing(as_frame=True)
X_housing = housing.data  
y_housing = housing.target  
df_housing = pd.concat([X_housing, y_housing.rename('MedHouseVal')], axis=1)  


In [ ]:
from sklearn.datasets import fetch_california_housing


Basic Information Pulls

In [ ]:
df_digits.head()

In [ ]:
df_housing.head()

In [ ]:
df_digits.info()

In [ ]:
df_housing.info()

In [ ]:
df_digits.describe()

In [ ]:
df_housing.describe()

Check for missing values and duplicates

In [ ]:
missing_values = df_digits.isnull().sum()
print("\nMissing values in each column:\n", missing_values[missing_values > 0])

print(f'\nDuplicates: {df_digits.duplicated().sum()}')


In [ ]:
missing_values = df_housing.isnull().sum()
print("\nMissing values in each column:\n", missing_values)

--Classification EDA Work--
-Plotting value counts and distribution of pixels

In [ ]:
target_counts = df_digits['Target'].value_counts()

plt.figure(figsize=(8, 5))
sns.barplot(x=target_counts.index, y=target_counts.values, palette='viridis')
plt.title("Distribution of Target Digits")
plt.xlabel("Digits")
plt.ylabel("Frequency")
plt.show()

Create correlation matrix and heatmap

In [ ]:

correlation_matrix = df_digits.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix.iloc[:20, :20], annot=False, cmap='coolwarm', cbar=True, vmin=-1, vmax=1)
plt.title("Correlation Heatmap (First 20 Pixels)")
plt.show()

Calculating Feature Variance

In [ ]:
feature_variances = df_digits.var().drop('Target')


plt.figure(figsize=(10, 5))
sns.lineplot(x=range(len(feature_variances)), y=feature_variances, color='blue')
plt.title("Feature Variances")
plt.xlabel("Feature Index")
plt.ylabel("Variance")
plt.show()


low_variance_features = feature_variances[feature_variances < 0.1]
print("\nLow-variance features:\n", low_variance_features)

Dimensionality Reduction using PCA

In [ ]:



pca = PCA(n_components=2)
digits_pca = pca.fit_transform(digits.data)

plt.figure(figsize=(10, 6))
sns.scatterplot(x=digits_pca[:, 0], y=digits_pca[:, 1], hue=digits.target, palette='tab10', s=60)
plt.title("2D PCA Projection of Digits Dataset")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.legend(title="Digits", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

Data Filtering to Isolate '0'

In [ ]:

digit_zero = df_digits[df_digits['Target'] == 0]


plt.figure(figsize=(12, 6))
for i in range(5):
    sns.kdeplot(digit_zero[f'Pixel_{i}'], label=f'Pixel_{i}', fill=True)
plt.title("Distribution of Pixel Values for Digit '0'")
plt.xlabel("Pixel Value")
plt.ylabel("Density")
plt.legend()
plt.show()

Pixel Pairplotting

In [ ]:

selected_pixels = ['Pixel_0', 'Pixel_10', 'Pixel_20', 'Pixel_30', 'Target']
sns.pairplot(df_digits[selected_pixels], hue='Target', palette='tab10')
plt.suptitle("Pairplot of Selected Pixels", y=1.02, fontsize=14)
plt.show()

Pixel Intensity Heatmap

In [ ]:

mean_pixel_intensity = df_digits.groupby('Target').mean().iloc[:, :64]


fig, axes = plt.subplots(2, 5, figsize=(12, 6))
for i, ax in enumerate(axes.ravel()):
    digit_heatmap = mean_pixel_intensity.iloc[i].values.reshape(8, 8)
    sns.heatmap(digit_heatmap, ax=ax, cmap='viridis', cbar=False, annot=False)
    ax.set_title(f"Digit: {i}")
    ax.axis('off')
plt.title("Average Pixel Intensity for Each Digit", fontsize=16)
plt.show()

--Regression EDA Work--


Univariate and Bivariate Analysis

In [ ]:
def UnivariateAnalysis(df):
    columns = df.columns
    n = len(columns)
    fig, axes = plt.subplots(n, 1, figsize=(8, 6 * n))
    for i, col in enumerate(columns):
        sns.histplot(df[col], kde=True, bins=10, ax=axes[i])
        axes[i].set_title(f'Histogram for {col}')
    plt.tight_layout()
    plt.show()


    fig, axes = plt.subplots(n, 1, figsize=(8, 6 * n))
    for i, col in enumerate(columns):
        sns.boxplot(x=df[col], ax=axes[i])
        axes[i].set_title(f'Boxplot for {col}')
    plt.tight_layout()
    plt.show()
print(cal_df.shape)
print(cal_df['Latitude'].nunique())
print(cal_df['Longitude'].nunique())
print(cal_df.head())
print(cal_df.isnull().sum())

UnivariateAnalysis(df_housing)

In [ ]:
def BivariateAnalysis(df):
    Pearson=df.corr(method='pearson')
    Spearman=df.corr(method='spearman')
    plt.figure(figsize=(8,6))
    sns.heatmap(Pearson, annot=True)
    plt.title("Pearson Correlation Index")
    plt.show()
    plt.figure(figsize=(8,6))
    sns.heatmap(Spearman, annot=True, cmap='coolwarm')
    plt.title('Spearman Correlation Index')
    plt.show()
BivariateAnalysis(df_housing)

Pairplots demonstrating positive and negative correlation between variables

In [ ]:
def PairPlots(df):
  sns.pairplot(df, kind='reg', plot_kws={'line_kws':{'color': 'red'}})
  plt.show()
  
selected=df_housing.drop(columns=['Latitude','Longitude','Population','Medvalue'])
PairPlots(selected)

Population=df_housing[['Population','Medvalue','MedInc','AveRooms','AveOccup']]
sns.pairplot(Population,kind='reg', plot_kws={'line_kws':{'color': 'red'}})
plt.show()

Outlier Detection Methods and Function comparing each

In [ ]:
def ZOutliers(df):
  zdf=df.copy()
  outliers=pd.DataFrame()
  for col in zdf.columns:
    zscore=stats.zscore(zdf[col])
    zdf[f'{col}_zscore']=zscore
    mask=zdf[f'{col}_zscore'].abs()<2.5
    zdf=zdf[mask]
    # outliers.drop_duplicates(inplace=True)
    # z_score_columns=[col for col in outliers.columns if col.endswith('_zscore')]
    # outliers=outliers.drop(columns=z_score_columns)
    # outliers.to_csv('outliers.csv', index=False)
    zdf.drop(columns=[f'{col}_zscore'], inplace=True)

  return zdf
def OutlierDetection(df):
  clean_df=df.copy()
  outliers=pd.DataFrame()
  for col in clean_df.columns:
    Q1=df[col].quantile(0.25)
    Q3=df[col].quantile(0.75)
    IQR=Q3-Q1
    lower=Q1-1.5*IQR
    upper=Q3+1.5*IQR
    mask=(df[col] >= lower) & (df[col]<=upper)
    clean_df=clean_df[mask]
  return clean_df
def KNNOutliers(df):
  df1=df.copy()
  combinedoutliers=pd.DataFrame()
  for col in df.columns:
    k=20
    knn=KNeighborsRegressor(n_neighbors=k)
    knn.fit(df1.iloc[:,1:9], df1[col])
    distances,indeces=knn.kneighbors(df1.iloc[:,1:9])
    df1[f'outlierscore for {col}']=np.mean(distances, axis=1)
    threshold=np.percentile(np.mean(distances, axis=1), 95)
    mask=df1[f'outlierscore for {col}']<threshold
    df1=df1[mask]
    df1.drop(columns=f'outlierscore for {col}', inplace=True)
  return df1

class OutlierHandling(BaseEstimator, TransformerMixin):
  def __init__(self, method='KNN'):
    self.method=method

  def fit(self, X, y=None):
    return self

  def transform(self, X):
    if self.method=='KNN':
      return KNNOutliers(X)
    elif self.method=='Z':
      return ZOutliers(X)
    elif self.method=='IQR':
      return OutlierDetection(X)
    else:
      raise ValueError("Invalid outlier detection method")
    
def outliertest(df):
  KNN=KNNOutliers(df)
  Z=ZOutliers(df)
  IQR=OutlierDetection(df)
  Outliers=[KNN,Z,IQR]
  regressionresults={}
  for i, outlier in enumerate(Outliers):
    outlier_indeces=outlier.index.tolist()
    outlier_indeces=[idx for idx in outlier_indeces if idx in df.index]
    cleandf=df.drop(index=outlier_indeces)
    X=cleandf.iloc[:,1:9].values
    y=cleandf['Medvalue'].values
    x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=42)
    clf=RandomForestRegressor()
    CLFDT=DecisionTreeRegressor()
    clf.fit(x_train,y_train)
    CLFDT.fit(x_train,y_train)
    y_pred=clf.predict(x_test)
    y_predDT=CLFDT.predict(x_test)
    print(f"Mean Squared Error for Random Forest for {i + 1}: {mean_squared_error(y_test,y_pred)}")
    print(f"R2 Score for Random Forest for {i + 1}: {r2_score(y_test,y_pred)}")
    print(f"Mean Squared Error for Decision Tree for {i + 1}: {mean_squared_error(y_test,y_predDT)}")
    print(f"R2 Score for Decision Tree for {i + 1}: {r2_score(y_test,y_predDT)}")
    regressionresults[f"Outlier Method {i + 1}"] = {
        "RandomForest": {
            "Mean Squared Error": mean_squared_error(y_test, y_pred),
            "R2 Score": r2_score(y_test, y_pred)
        },
        "DecisionTree": {
            "Mean Squared Error": mean_squared_error(y_test, y_predDT),
            "R2 Score": r2_score(y_test, y_predDT)
        }
    }
  random_mse=[regressionresults[method]['RandomForest']['Mean Squared Error'] for method in regressionresults]
  random_r2=[regressionresults[method]['RandomForest']['R2 Score'] for method in regressionresults]
  decision_mse=[regressionresults[method]['DecisionTree']['Mean Squared Error'] for method in regressionresults]
  decision_r2=[regressionresults[method]['DecisionTree']['R2 Score'] for method in regressionresults]
  fig,ax=plt.subplots(2,2, figsize=(12,12))
  plt.bar(['IQR','KNN','ZOutliers'],random_mse,width=1, edgecolor='red', color='blue')
  plt.title('Random Forest Mean Squared Error')
  plt.subplot(2,2,1)
  plt.bar(['IQR','KNN','ZOutliers'],random_r2,width=1, edgecolor='red', color='blue')
  plt.title('Random Forest R2 Score')
  plt.subplot(2,2,2)
  plt.bar(['IQR','KNN','ZOutliers'],decision_mse,width=1, edgecolor='red', color='blue')
  plt.title('Decision Tree Mean Squared Error')
  plt.subplot(2,2,3)
  plt.bar(['IQR','KNN','ZOutliers'],decision_r2,width=1, edgecolor='red', color='blue')
  plt.title('Decision Tree R2 Score')
  plt.subplot(2,2,4)
  plt.show()
outliertest(df_housing)

Histograms for House Age

In [ ]:
plt.figure(figsize=(8, 5))
sns.histplot(df_housing['HouseAge'], kde=True, color='green', bins=30)
plt.title("Distribution of House Age")
plt.xlabel("House Age (Years)")
plt.ylabel("Frequency")
plt.show()

Log Transformation of Median House Value

In [ ]:

df_housing['MedHouseVal_log'] = np.log1p(df_housing['MedHouseVal'])

plt.figure(figsize=(8, 5))
sns.histplot(df_housing['MedHouseVal_log'], kde=True, color='blue', bins=30)
plt.title("Log-Transformed Distribution of Median House Value")
plt.xlabel("Log Median House Value ($100,000)")
plt.ylabel("Frequency")
plt.show()

Plot the boxplot of the log-transformed median house value by median income quintiles


In [ ]:


df_housing['MedInc_quintile'] = pd.qcut(df_housing['MedInc'], 5)

plt.figure(figsize=(10, 6))
sns.boxplot(x='MedInc_quintile', y='MedHouseVal_log', data=df_housing, palette="cool")
plt.title("Log-Transformed Median House Value by Median Income Quintiles")
plt.xlabel("Median Income (Quintiles)")
plt.ylabel("Log Median House Value ($100,000)")
plt.xticks(rotation=45)
plt.show()

Watson's GeoPandas Work

Configure Utilities

In [ ]:

np.set_printoptions(suppress=True, precision=5)
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

Load Data

In [ ]:
'''Load Data'''
common_crs = "EPSG:4269"
coastline_shp = './geoData/califorina_pacific_coast-shapefile/3853-s3_2002_s3_reg_pacific_ocean.shp'
places_shp = './geoData/ca_places/CA_Places.shp'
counties_shp = './geoData/us_county/tl_2023_us_county.shp'
# counties_shp = './geoData/ca_counties/CA_Counties.shp'


coastline = gpd.read_file(coastline_shp)
places = gpd.read_file(places_shp)
counties = gpd.read_file(counties_shp)

rawData = fetch_california_housing(as_frame=True)
cal_housing_geo = gpd.GeoDataFrame(rawData.data, geometry=gpd.points_from_xy(rawData.data.Longitude, rawData.data.Latitude), crs=common_crs)
cal_housing_geo['y'] = rawData.target

Standardize Coordinate Projection

In [ ]:
coastline = coastline.to_crs(common_crs)
places = places.to_crs(common_crs)
counties = counties.to_crs(common_crs)

In [ ]:
Set Geo Bounding Box and Plot


housing_bounds = cal_housing_geo.total_bounds
counties = counties.clip(housing_bounds)
coastline = coastline.clip(housing_bounds)
fig, ax = plt.subplots(1, 1, figsize=(8, 6), tight_layout=True)

coastline.plot(ax=ax, edgecolor='blue', linewidth=0.1)
counties.boundary.plot(ax=ax, edgecolor='black', linewidth=.5)
cal_housing_geo.plot(ax=ax, color='red', linewidth=0.1)
places.plot(ax=ax, color='green', linewidth=1.0)
plt.show()

Feature Engineer From Coast

In [ ]:
house_to_coast = gpd.sjoin_nearest(cal_housing_geo, coastline, how="left", distance_col="meters")
cal_housing_geo['m_to_coast'] = house_to_coast['meters']

Feature Engineer Distance from Census Designated Place

In [ ]:

house_to_interest = gpd.sjoin_nearest(cal_housing_geo, places, how="left", distance_col="meters")
# cal_housing_geo['CDP'] = house_to_interest['NAME'] // The name of the CDP does not appear to impact model performance
cal_housing_geo['m_to_CDP'] = house_to_interest['meters']

Feature Engineer County Data

In [ ]:

# This data point does not appear to impact model performance, but can be useful for EDA.
# house_to_county = gpd.sjoin_nearest(cal_housing_geo, counties, how="left")
# cal_housing_geo['county_name'] = house_to_county['NAME']

Color Code Counties by Target Value

In [ ]:
edaData = gpd.GeoDataFrame(rawData.data, geometry=gpd.points_from_xy(rawData.data.Longitude, rawData.data.Latitude), crs=common_crs)
edaData['y'] = rawData.target

house_to_county = gpd.sjoin_nearest(edaData, counties, how="left")
edaData['county_name'] = house_to_county['NAME']

meanTarget = edaData.groupby('county_name')['y'].mean().reset_index()

print(meanTarget[meanTarget['county_name'] == 'Alameda']['y'][0])

Add Mean Target Value to County Data

In [ ]:
meanTarget = edaData.groupby('county_name')['y'].mean().reset_index()

def setMeanTarget(row):
    value = meanTarget[meanTarget['county_name'] == row['NAME']]['y'].array
    row['mean_target'] = value[0] if len(value) else None
    return row

withTarget = counties.apply(setMeanTarget, axis=1)
withTarget.dropna(subset='mean_target')

withTarget.plot(column='mean_target', legend=True)
plt.title('Median Home Value')

# cal_housing_geo = encode(cal_housing_geo, ['CDP','county_name'])

Run Default Regressor Timeline to determine Feature Importance

In [ ]:
modelData = cal_housing_geo.drop(columns=['y', 'geometry'])
X_train, X_test, y_train, y_test = train_test_split(modelData, rawData.target, test_size=0.3, random_state=42)

pipe = Pipeline([('scaler', StandardScaler()),('regress',RandomForestRegressor())], verbose=True)
predicted = pipe.fit(X_train, y_train).predict(X_test)

mse = mean_squared_error(y_test, predicted)
print(mse)

regressor = pipe.steps[-1][-1]

feat_importance = list(zip(modelData.columns,regressor.feature_importances_))
feat_importance = sorted(feat_importance, key=lambda tup: tup[1])

feat=[x for x,y in feat_importance]
importance=[y for x,y in feat_importance]


plt.title('Feature Importances')
sns.barplot(y=feat, x=importance)

Mike's GeoPandas Work

Load School District Information and plot 

In [ ]:
schoolsfile='/content/drive/MyDrive/schools.shp'
os.environ['SHAPE_RESTORE_SHX'] = 'YES'
schools=gpd.read_file(schoolsfile)
schools['unique_id'] = range(1, len(schools) + 1)

print(schools.info())
print(schools.nunique())


housing=fetch_california_housing()
cal_df=pd.DataFrame(data=housing.data, columns=housing.feature_names)
cal_df['Medvalue'] = housing.target

zip latitude and longitude together to facilitate sJoin

In [ ]:
latlong = [Point(xy) for xy in zip(cal_df['Longitude'], cal_df['Latitude'])]
new_df = gpd.GeoDataFrame(cal_df, geometry=latlong)

Plot xy coordinates over district map

In [ ]:
ax=schools.plot(figsize=(10,10), alpha=0.5, edgecolor='k')
new_df.plot(ax=ax, column='Medvalue', cmap='coolwarm', markersize=5, legend=True)
plt.title('California Housing Prices and Schools')
plt.show()

SJoin Cal Housing dataset with School districts, and build new features based upon median income per district and population density for each district.

In [ ]:
joineddf=gpd.sjoin(schools, new_df, how='left', predicate='intersects')

# print(joineddf.nunique())
# print(joineddf.isnull().sum())
# print(joineddf.duplicated(subset='unique_id').sum())


joineddf['schooldistrictmedianincome']=joineddf.groupby('unique_id')['MedInc'].transform('mean')
joineddf['populationdensity']=joineddf.groupby('unique_id')['Population'].transform('mean')
joineddf=joineddf.dropna()
print(joineddf.info())

Run each feature over the district map

In [ ]:
ax=schools.plot(figsize=(10,10), alpha=0.5, edgecolor='k')
joineddf.plot(ax=ax, column='schooldistrictmedianincome', cmap='coolwarm', markersize=5, legend=True)
plt.title('California Median Income aligned with School Districts')
plt.show()
ax=schools.plot(figsize=(10,10), alpha=0.5, edgecolor='k')
joineddf.plot(ax=ax, column='populationdensity', cmap='coolwarm', markersize=5, legend=True)
plt.title('California School Districts aligned with average population')
plt.show()

Run default Decision Tree and Random Forest Classifiers to determine feature importance

In [ ]:
def RegressionAnalysis(df):
  df=joineddf.drop(columns=['Medvalue','geometry'])
  X=df.values
  y=joineddf['Medvalue'].values
  scaler = StandardScaler()
  X=scaler.fit_transform(X)
  x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=42)
  clf=RandomForestRegressor()
  CLFDT=DecisionTreeRegressor()
  clf.fit(x_train,y_train)
  CLFDT.fit(x_train,y_train)
  y_pred=clf.predict(x_test)
  y_predDT=CLFDT.predict(x_test)
  print(f"Mean Squared Error for Random Forest: {mean_squared_error(y_test,y_pred)}")
  print(f"R2 Score for Random Forest: {r2_score(y_test,y_pred)}")
  print(f"Mean Squared Error for Decision Tree: {mean_squared_error(y_test,y_predDT)}")
  print(f"R2 Score for Decision Tree: {r2_score(y_test,y_predDT)}")
  importances = clf.feature_importances_
  feature_imp_df = pd.DataFrame({'Feature': df.columns, 'Gini Importance': importances}).sort_values('Gini Importance', ascending=False) 
  print(feature_imp_df)
  plt.figure(figsize=(8, 4))
  plt.barh(df.columns, importances, color='skyblue')
  plt.xlabel('Feature Importance')
  plt.title('Feature Importance')
  plt.gca().invert_yaxis()  
  plt.show()
  return y_pred,y_test,y_predDT
  

RegressionAnalysis(joineddf)